In [1]:
import QuantLib as ql
from utilities.Enums import Currency, SwapDirection
from instruments.interestRateInstrument.interestRateSwap import InterestRateSwap
from instruments.interestRateInstrument.swaption import Swaption
from marketdata import init_marketdata

In [2]:
irs = InterestRateSwap(notional = 1000000, 
                       currency = Currency.USD, 
                       timeToSwapStart=2, 
                       timeToSwapEnd=11, 
                       swapDirection=SwapDirection.RECEIVER,
                       fixed_rate=0.025, 
                       float_spread = 0.004)

In [3]:
exerciseDate = init_marketdata.today + ql.Period(1, ql.Years)
exercise = ql.EuropeanExercise(exerciseDate)
swaption = ql.Swaption(irs.ql_swap, exercise)

In [4]:
from marketdata.util import today, calendar, day_count, business_day_convention
yts = ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.02)), day_count)
yts_handle = ql.YieldTermStructureHandle(yts)

Pricing swaption using Black 76 and a single quote volatility

In [5]:
vol_handle = ql.QuoteHandle(ql.SimpleQuote(0.2))
single_quote_engine = ql.BlackSwaptionEngine(yts_handle, vol_handle)

In [6]:
swaption.setPricingEngine(single_quote_engine)

In [7]:
swaption.NPV()

15971.59884786479

Pricing swaption using Black 76 and a ``ConstantSwaptionVolatility``

In [8]:
flat_surface_handle = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(today, calendar, business_day_convention, ql.QuoteHandle(ql.SimpleQuote(0.2)), day_count))
flat_surface_engine = ql.BlackSwaptionEngine(yts_handle, flat_surface_handle)

In [9]:
swaption.setPricingEngine(flat_surface_engine)
swaption.NPV()

15971.59884786479

Pricing swaption using Black 76 and a Swaption Vol Surface

In [10]:
optionTenors = [ql.Period(1, ql.Months),
                ql.Period(3, ql.Months),
                ql.Period(6, ql.Months),
                ql.Period(1, ql.Years),
                ql.Period(2, ql.Years)]
swapTenors = [ql.Period(1, ql.Months),
              ql.Period(2, ql.Months),
              ql.Period(3, ql.Months),
              ql.Period(20, ql.Years)]
vols = [[ql.QuoteHandle(ql.SimpleQuote(0.2)),
         ql.QuoteHandle(ql.SimpleQuote(0.2)),
         ql.QuoteHandle(ql.SimpleQuote(0.2)),
         ql.QuoteHandle(ql.SimpleQuote(0.2))],
        [ql.QuoteHandle(ql.SimpleQuote(0.3)),
         ql.QuoteHandle(ql.SimpleQuote(0.3)),
         ql.QuoteHandle(ql.SimpleQuote(0.3)),
         ql.QuoteHandle(ql.SimpleQuote(0.3))],
        [ql.QuoteHandle(ql.SimpleQuote(0.4)),
         ql.QuoteHandle(ql.SimpleQuote(0.4)),
         ql.QuoteHandle(ql.SimpleQuote(0.4)),
         ql.QuoteHandle(ql.SimpleQuote(0.4))],
        [ql.QuoteHandle(ql.SimpleQuote(0.5)),
         ql.QuoteHandle(ql.SimpleQuote(0.5)),
         ql.QuoteHandle(ql.SimpleQuote(0.5)),
         ql.QuoteHandle(ql.SimpleQuote(0.5))],
        [ql.QuoteHandle(ql.SimpleQuote(0.6)),
         ql.QuoteHandle(ql.SimpleQuote(0.6)),
         ql.QuoteHandle(ql.SimpleQuote(0.6)),
         ql.QuoteHandle(ql.SimpleQuote(0.6))],
        ]
swaptionVolatilityMatrix = ql.SwaptionVolatilityMatrix(calendar, business_day_convention, optionTenors, swapTenors, vols, day_count)

In [11]:
real_surface_handle = ql.SwaptionVolatilityStructureHandle(swaptionVolatilityMatrix)
real_surface_engine = ql.BlackSwaptionEngine(yts_handle, real_surface_handle)

In [12]:
swaption.setPricingEngine(real_surface_engine)
swaption.NPV()

32575.562556741268